## Radon Transform

$R(\rho, \theta) = \int_{-\infty}^{+\infty} \int_{-\infty}^{+\infty} I(x, y) \, \delta(x \cos \theta + y \sin \theta - \rho) \, dx \, dy$

- $I(x, y)$ is the image being transformed.
- $\delta$ is the Dirac distribution.
- $\rho$ is the distance from the origin to the line along which the function \( I \) is integrated.
- $\theta$ is the angle between the x-axis and the normal to the line.

##### Load the necessary modules

In [1]:
import numpy as np
from skimage.transform import rotate as skimage_rotate

##### Define Radon Transform

In [2]:
def RT(img, nb_angles):
    if img.ndim != 2:
        raise ValueError('The input image must be 2-D')
        
    thetas = np.linspace(0, 180, nb_angles, endpoint=False)
    rotations = np.array([skimage_rotate(img, -1*theta) for theta in thetas])
    sinogram = np.array([rotation.sum(axis=0) for rotation in rotations]).T
    
    # Suivre l'exécution
    progress = (k + 1) / nb_angles * 100
    print(f'\rRT progress: ({progress:.2f}%)', end='')

    return sinogram